In [1]:
from pynwb.spec import NWBDatasetSpec
from pynwb.spec import NWBNamespaceBuilder, NWBGroupSpec, NWBAttributeSpec

name = 'ecog'
ns_path = name + ".namespace.yaml"
ext_source = name + ".extensions.yaml"


ns_builder = NWBNamespaceBuilder(name + ' extensions', name, version='0.1.0')

slm = NWBGroupSpec(
        neurodata_type_def='SpatialLightModulator',
        neurodata_type_inc='NWBContainer',
        doc=('SpatialLightModulator'),
        attributes=[NWBAttributeSpec('dimensions', 'dimensions ([w, h] or [w, h, d]) of SLM field', 'numeric', shape=((2, ), (3,)))
        ]
)
ns_builder.add_spec(ext_source, slm)
ns_builder.export(ns_path)

from pynwb import load_namespaces, get_class, NWBHDF5IO, NWBFile
from datetime import datetime
import numpy as np



In [4]:
from pynwb import register_class
from pynwb.core import NWBContainer
from collections.abc import Iterable
from hdmf.utils import docval, popargs
from hdmf.utils import docval, get_docval, popargs
from hdmf.utils import docval, popargs, get_docval

load_namespaces('ecog.namespace.yaml')
def popargs_to_dict(keys, argdict):
    return {key: argdict.pop(key) for key in keys}

@register_class('SpatialLightModulator', 'ecog')
class SpatialLightModulator(NWBContainer):
    __nwbfields__ = ('dimensions', 'dimensions1')

    @docval(
        *get_docval(NWBContainer.__init__) + (
        {'name': 'dimensions', 'type': Iterable, 'doc': 'dimensions ([w, h] or [w, h, d]) of SLM field'},
        {'name': 'dimensions1', 'type': Iterable, 'doc': 'dimensions ([w, h] or [w, h, d]) of SLM field'})
    )
    def __init__(self, **kwargs):
        load_namespaces('ecog.namespace.yaml')
        args_to_set = popargs_to_dict(('dimensions', 'dimensions1'), kwargs)
        super().__init__(**kwargs)
        for key, val in args_to_set.items():
            setattr(self, key, val)


slm = SpatialLightModulator(
    name="slm",
    dimensions = [1, 2, 3, 5],
    dimensions1 = [1, 2,3, 4, 5]
)

ValueError: No specification for 'SpatialLightModulator1' in namespace 'ecog'

In [5]:
SpatialLightModulator = get_class('SpatialLightModulator', 'ecog')

slm = SpatialLightModulator(
    name="slm",
    dimensions = [1, 2, 3, 4, 5]
)

TypeError: SpatialLightModulator.__init__: missing argument 'dimensions1'

In [20]:
print(slm)

{'doc': 'SpatialLightModulator', 'neurodata_type_inc': 'NWBContainer', 'neurodata_type_def': 'SpatialLightModulator', 'attributes': [{'doc': 'dimensions ([w, h] or [w, h, d]) of SLM field', 'name': 'dimensions', 'dtype': 'numeric', 'shape': ((2,), (3,))}]}


In [1]:
from pynwb.spec import NWBNamespaceBuilder, NWBGroupSpec, NWBAttributeSpec

name = 'test_multicontainerinterface'
ns_path = name + ".namespace.yaml"
ext_source = name + ".extensions.yaml"

ns_builder = NWBNamespaceBuilder(name + ' extensions', name, version='0.1.0')
ns_builder.include_type('NWBDataInterface', namespace='core')

potato = NWBGroupSpec(neurodata_type_def='Potato',
                      neurodata_type_inc='NWBDataInterface',
                      doc='A potato', quantity='*',
                      attributes=[
                          NWBAttributeSpec(name='weight',
                                           doc='weight of potato',
                                           dtype='float',
                                           shape=(None,2),
                                           required=True),
                          NWBAttributeSpec(name='age',
                                           doc='age of potato',
                                           dtype='float',
                                           required=False)
                      ])

potato_sack = NWBGroupSpec(neurodata_type_def='PotatoSack',
                           neurodata_type_inc='NWBDataInterface',
                           name='potato_sack',
                           doc='A sack of potatoes', quantity='?',
                           groups=[potato])

ns_builder.add_spec(ext_source, potato_sack)
ns_builder.export(ns_path)

In [4]:
from pynwb import register_class, load_namespaces
from pynwb.file import MultiContainerInterface, NWBContainer
from pynwb import register_class
from pynwb.core import NWBContainer
from collections.abc import Iterable
from hdmf.utils import docval, popargs
from hdmf.utils import docval, get_docval, popargs
from hdmf.utils import docval, popargs, get_docval, popargs_to_dict
from collections.abc import Iterable
import numpy as np


load_namespaces(ns_path)

@register_class('Potato', name)
class Potato(NWBContainer):
    __nwbfields__ = ('name', 'weight', 'age')

    @docval({'name': 'name', 'type': str, 'doc': 'who names a potato?'},
            {'name': 'weight', 'type': (float, Iterable), 'doc': 'weight of potato in grams'},
            {'name': 'age', 'type': float, 'doc': 'age of potato in days'})
    def __init__(self, **kwargs):

        super().__init__(name=kwargs['name'])
        self.weight = kwargs['weight']
        self.age = kwargs['age']


@register_class('PotatoSack', name)
class PotatoSack(MultiContainerInterface):

    __clsconf__ = {
        'attr': 'potatos',
        'type': Potato,
        'add': 'add_potato',
        'get': 'get_potato',
        'create': 'create_potato',
    }

In [5]:
from pynwb import NWBHDF5IO, NWBFile
from datetime import datetime
from dateutil.tz import tzlocal

# You can add potatoes to a potato sack in different ways
potato_sack = PotatoSack(potatos=Potato(name='potato1', age=2.3, weight=3.0))
potato_sack.add_potato(Potato('potato2', [3.0, 1], 4.0))
potato_sack.create_potato('big_potato', np.array([3.5, 5]), 20.0)

nwbfile = NWBFile("a file with metadata", "NB123A", datetime(2018, 6, 1, tzinfo=tzlocal()))

pmod = nwbfile.create_processing_module('module_name', 'desc')
pmod.add_container(potato_sack)